# NNodely Documentation - State Variables

In [8]:
# uncomment the command below to install the nnodely package
#!pip install nnodely

from nnodely import *
from nnodely.relation import NeuObj

## State Definition

Use the keyword 'State' to define a state variable (like you will do for an Input). You can specify the dimension of the variable.

In [9]:
x_state = State('x_state', dimensions=1)
x_out = Fir(x_state.tw(0.5))

## Closed Loop

Every relation inside NNodely can update a state variable. closing a state in a loop means that at the end of each forward pass the result of the selected relation will update the selected state variable.

In [11]:
NeuObj.clearNames()
x_out.closedLoop(x_state)
out = Output('out',x_out)

or you can use the ClosedLoop block

In [13]:
NeuObj.clearNames()
x_out = ClosedLoop(x_out, x_state)
out = Output('out',x_out)

or you can use the NNodely framework directly

In [14]:
model = Modely()
model.addClosedLoop(x_out, x_state)

## Connect

Every relation inside NNodely can update a state variable. connecting a relation to a state means that at each forward pass the result of the selected relation will immediately update the selected state variable.

(Note: you must re-define the relation in order to change the update of the state variable)

In [17]:
NeuObj.clearNames()
x_out = Fir(x_state.tw(0.5))
x_out.connect(x_state)
out = Output('out',x_out)

or you can use the Connect block

In [18]:
NeuObj.clearNames()
x_out = Connect(x_out, x_state)
out = Output('out',x_out)

or you can use the NNodely framework directly

In [19]:
model = Modely()
model.addConnect(x_out, x_state)

## Connect two models together

In [ ]:
NeuObj.clearNames()
import numpy as np

def linear_function(x, k1, k2):
    return x*k1 + k2

data_a = np.arange(1,101, dtype=np.float32)
data_b_t = linear_function(data_a, 2, 3)

data_c = np.arange(1,101, dtype=np.float32)
data_b_in = np.arange(5,105, dtype=np.float32)
data_d_t = linear_function(data_c, 5, 1)

dataset = {'a': data_a, 'b_t': data_b_t, 'c':data_c, 'b_in': data_b_in, 'd_t':data_d_t }
## Model a
a = Input('a')
b_t = Input('b_t')
shared = Parameter('shared',dimensions=(1,1))
b = Output('b',Linear(W=shared)(a.last())+Linear(W='A')(Fir(W='B')(a.tw(0.5))))

model = Modely(seed=42)
model.addModel('b_model', b)
model.addMinimize('b_min', b, b_t.last())
model.neuralizeModel(0.1)

# Model d
c = Input('c')
d_t = Input('d_t')
b_in = State('b_in')
model.addConnect(b, b_in)
d = Output('d',Linear(W=shared)(c.last())+Fir(W='C')(c.tw(0.5))+Fir(W='D')(b_in.tw(0.3)))

model.addModel('d_model', [b,d])
model.addMinimize('d_min', d, d_t.last())
model.neuralizeModel(0.1)
model.loadData('dataset', dataset)

params = {'num_of_epochs': 1,
        'train_batch_size': 8,
        'val_batch_size': 8,
        'test_batch_size':1,
        'lr':0.1}

## training dei parametri di tutti i modelli
model.trainModel(splits=[100,0,0], training_params=params, prediction_samples=4)
print('connect variables: ', model.model.connect_update)
print('closed loop variables: ', model.model.closed_loop_update)

## Recurrent Train

In order to do a recurrent training of the network using the State variables is mandatory to specify the window of prediction (prediction_samples).

In [33]:
import numpy as np
NeuObj.clearNames()
x = Input('x', dimensions=3)
x_state = State('x_state', dimensions=3)
y_state = State('y_state', dimensions=3)
x_out = Linear(output_dimension=3)(x_state.tw(0.5))
y_out = Linear(output_dimension=3)(y_state.tw(0.5))
x_out.closedLoop(x_state)
y_out.closedLoop(y_state)
out = Output('out',x_out+y_out)

test = Modely(seed=42)
test.addModel('model', out)
test.addMinimize('error', out, x.tw(0.5))

test.neuralizeModel(0.1)

dataset = {'x':np.random.uniform(1,4,200)}
test.loadData(name='dataset', source=dataset)

# Training non ricorrente
params = {'num_of_epochs': 1, 'train_batch_size': 4, 'val_batch_size':4, 'test_batch_size':1, 'lr':0.01}
test.trainModel(splits=[70,20,10], prediction_samples=3, shuffle_data=False, training_params=params)
print('finale state: ', test.states)

================================ nnodely Model =================================
{'Constants': {},
 'Functions': {},
 'Info': {'SampleTime': 0.1,
          'nnodely_version': '1.3.1',
          'ns': [5, 0],
          'ntot': 5,
          'num_parameters': 18},
 'Inputs': {'x': {'dim': 3,
                  'ns': [5, 0],
                  'ntot': 5,
                  'sw': [0, 0],
                  'tw': [-0.5, 0]}},
 'Minimizers': {'error': {'A': 'out', 'B': 'TimePart100', 'loss': 'mse'}},
 'Models': 'model',
 'Outputs': {'out': 'Add98'},
 'Parameters': {'PLinear3W': {'dim': [3, 3]}, 'PLinear5W': {'dim': [3, 3]}},
 'Relations': {'Add98': ['Add', ['Linear94', 'Linear97']],
               'Linear94': ['Linear', ['TimePart93'], 'PLinear3W', None, 0],
               'Linear97': ['Linear', ['TimePart96'], 'PLinear5W', None, 0],
               'TimePart100': ['TimePart', ['x'], -1, [-0.5, 0]],
               'TimePart93': ['TimePart', ['x_state'], -1, [-0.5, 0]],
               'TimePart96':

c:\Users\User\Desktop\VSCodeProjects\nnodely\nnodely\venv\Lib\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([4, 5, 1])) that is different to the input size (torch.Size([4, 5, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\User\Desktop\VSCodeProjects\nnodely\nnodely\venv\Lib\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([4, 5, 1])) that is different to the input size (torch.Size([4, 5, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\User\Desktop\VSCodeProjects\nnodely\nnodely\venv\Lib\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([1, 5, 1])) that is different to the input size (torch.Size([1, 5, 3])). 

## Clear State

use the specific function to manually clear the state of a state variable

In [34]:
test.resetStates()
print('finale state: ', test.states)

finale state:  {'y_state': tensor([[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]), 'x_state': tensor([[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]])}


## State update at training time

States variables can also be created at training time. These variables will exist only during the training process.

To create the you have to define a dictionary containing {input:output}. In that case, the 'input' will become a state during the training process and the 'output' will be the relation updating it.

In [35]:
import numpy as np
NeuObj.clearNames()
x = Input('x', dimensions=3)
x_s = Input('x_s', dimensions=3)
y_s = Input('y_s', dimensions=3)
x_out = Linear(output_dimension=3)(x_s.tw(0.5))
y_out = Linear(output_dimension=3)(y_s.tw(0.5))
out = Output('out',x_out+y_out)
out_x = Output('out_x',x_out)
out_y = Output('out_y',y_out)

test = Modely(seed=42)
test.addModel('model', [out,out_x,out_y])
test.addMinimize('error', out, x.tw(0.5))

test.neuralizeModel(0.1)

dataset = {'x':np.random.uniform(1,4,100)}
test.loadData(name='dataset', source=dataset)

# Training non ricorrente
params = {'num_of_epochs': 10, 'train_batch_size': 4, 'val_batch_size':4, 'test_batch_size':1, 'lr':0.01}
test.trainModel(splits=[70,20,10], prediction_samples=3, shuffle_data=False, closed_loop={'x_s':'out_x','y_s':'out_y'}, training_params=params)
print('finale state: ', test.states)

================================ nnodely Model =================================
{'Constants': {},
 'Functions': {},
 'Info': {'SampleTime': 0.1,
          'nnodely_version': '1.3.1',
          'ns': [5, 0],
          'ntot': 5,
          'num_parameters': 18},
 'Inputs': {'x': {'dim': 3,
                  'ns': [5, 0],
                  'ntot': 5,
                  'sw': [0, 0],
                  'tw': [-0.5, 0]},
            'x_s': {'dim': 3,
                    'ns': [5, 0],
                    'ntot': 5,
                    'sw': [0, 0],
                    'tw': [-0.5, 0]},
            'y_s': {'dim': 3,
                    'ns': [5, 0],
                    'ntot': 5,
                    'sw': [0, 0],
                    'tw': [-0.5, 0]}},
 'Minimizers': {'error': {'A': 'out', 'B': 'TimePart109', 'loss': 'mse'}},
 'Models': 'model',
 'Outputs': {'out': 'Add107', 'out_x': 'Linear103', 'out_y': 'Linear106'},
 'Parameters': {'PLinear3W': {'dim': [3, 3]}, 'PLinear5W': {'dim': [3, 3]}},

[trainModel] Recurrent train: closing the loop between the the input ports y_s and the output ports out_y for 3 samples
======================== nnodely Model Train Parameters ========================
models:                       ['model']
num of epochs:                10
update per epochs:            16
└>(n_samples-batch_size-prediction_samples+1)/(batch_size+step-1)+1
prediction samples:           3
step:                         0
closed loop:                  {'x_s': 'out_x', 'y_s': 'out_y'}
connect:                      {}
train dataset:                train_dataset_0.70
	- num of samples:            67
	- batch size:                4
	- unused samples:            0
	  └>n_samples-prediction_samples-update_per_epochs*(batch_size+step-1)
val dataset:                  validation_dataset_0.20
val {batch size, samples}:    {4, 19}
test dataset:                 test_dataset_0.10
test {batch size, samples}:   {1, 10}
minimizers:                   {'error': {'A': 'out',
                

c:\Users\User\Desktop\VSCodeProjects\nnodely\nnodely\venv\Lib\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([4, 5, 1])) that is different to the input size (torch.Size([4, 5, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


|   4/10   |6.421e+00|6.608e+00|6.421e+00|6.608e+00|
|   5/10   |6.421e+00|6.608e+00|6.421e+00|6.608e+00|
|   6/10   |6.421e+00|6.608e+00|6.421e+00|6.608e+00|
|   7/10   |6.421e+00|6.608e+00|6.421e+00|6.608e+00|
|   8/10   |6.421e+00|6.608e+00|6.421e+00|6.608e+00|
|   9/10   |6.421e+00|6.608e+00|6.421e+00|6.608e+00|
|  10/10   |6.421e+00|6.608e+00|6.421e+00|6.608e+00|
|--------------------------------------------------|
============================ nnodely Training Time =============================
Total time of Training:       0.49455714225769043
The selected model is the LAST model of the training.
=========== nnodely Model Results for train_dataset_0.70 ==========
| Loss|        mse        |        FVU        |        AIC        |
|     |    small better   |    small better   |    lower better   |
|-----------------------------------------------------------------|
|error|     6.421e+00     |        inf        |     3.119e+04     |
|--------------------------------------------------

c:\Users\User\Desktop\VSCodeProjects\nnodely\nnodely\venv\Lib\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([4, 5, 1])) that is different to the input size (torch.Size([4, 5, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\User\Desktop\VSCodeProjects\nnodely\nnodely\venv\Lib\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([1, 5, 1])) that is different to the input size (torch.Size([1, 5, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


## Simple One Pole Convergence

In [36]:
NeuObj.clearNames()
x = Input('x')
xs = State('x_state')
int = Integrate((x.last()-xs.last()))
int.closedLoop(xs)
out = Output('pole',int)
integrator = Modely()
integrator.addModel('out', out)
integrator.neuralizeModel(0.1)

================================ nnodely Model =================================
{'Constants': {'SampleTime': {'dim': 1, 'values': 0.1}},
 'Functions': {},
 'Info': {'SampleTime': 0.1,
          'nnodely_version': '1.3.1',
          'ns': [1, 0],
          'ntot': 1,
          'num_parameters': 0},
 'Inputs': {'x': {'dim': 1,
                  'ns': [1, 0],
                  'ntot': 1,
                  'sw': [-1, 0],
                  'tw': [0, 0]}},
 'Minimizers': {},
 'Models': 'out',
 'Outputs': {'pole': 'Add119'},
 'Parameters': {},
 'Relations': {'Add119': ['Add', ['SamplePart116', 'Mul118']],
               'Mul118': ['Mul', ['Sub114', 'SampleTime']],
               'SamplePart111': ['SamplePart', ['x'], -1, [-1, 0]],
               'SamplePart113': ['SamplePart', ['x_state'], -1, [-1, 0]],
               'SamplePart116': ['SamplePart', ['Sub114_int2'], -1, [-1, 0]],
               'Sub114': ['Sub', ['SamplePart111', 'SamplePart113']]},
 'States': {'Sub114_int2': {'closedLoop': 